In [ ]:
!pip install faiss-cpu
!pip install faiss-gpu
!pip install gradio 
!pip install yolov5


In [ ]:
!pip install datasets==2.9

In [1]:
from objectdetection import ObjectDetection
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel
from PIL import Image
import requests
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms as T
import torch.nn as nn
from tqdm import tqdm
from matplotlib import pyplot as plt
import os
import numpy as np
import torch.nn.functional as F
from rerankForDemo import Initial, reranking
import gradio as gr
import pandas as pd

In [2]:
init = Initial("ShuffleNet")

Using cache found in /home/jupyter/.cache/torch/hub/pytorch_vision_v0.10.0


In [3]:
img1 = Image.open("data/img/CLOTHING/Blouse/id_00004848/comsumer_01.jpg")
img2 = Image.open("data/img/CLOTHING/Blouse/id_00004848/comsumer_01_blur.jpg")

In [4]:
reranking(img1, [img2], init)

[0.32882434129714966]

In [5]:
od = ObjectDetection()
#label, img, coor = od.detect(img1)

In [6]:
from bi_encoder import bi_encoder

In [7]:
bi = bi_encoder()

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from datasets import load_dataset
import faiss

bi.embeddings_dataset = load_dataset("vietdata/fashion_database")["train"]
bi.embeddings_dataset.add_faiss_index(column="embeddings", metric_type=faiss.METRIC_INNER_PRODUCT)

Using custom data configuration vietdata--fashion_database-09a37abf11bdeed9
Found cached dataset parquet (/home/jupyter/.cache/huggingface/datasets/vietdata___parquet/vietdata--fashion_database-09a37abf11bdeed9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

Dataset({
    features: ['embeddings', 'paths'],
    num_rows: 22669
})

In [9]:
def retrieve(img):
    related, scores = bi.search_images_from_image(img, topk=100)
    #print(scores)
    #sims = []
    #for i in related:
    #    sims.append(reranking(img, i, init))
    sims = reranking(img, related, init)
    indices = np.argsort(np.array(sims))
    #print(np.array(sims)[indices])
    return [(related[i], sims[i]) for i in indices[::-1][:5]], [(x, float(y)) for x, y in zip(related[:5],scores[:5])]

def object_retrieve(od_objects, label: gr.SelectData):
    labels, imgs, coors = od_objects
    print(label.value)
    for idx, label_ in enumerate(labels):
        if label_ == label.value:
            return retrieve(imgs[idx])
    return []

def products(img):
    labels, imgs, arr, coors = od.detect(img)
    print(labels)
    sections = [([int(i) for i in x], y) for x, y in zip(coors, labels)]
    return (img, sections), (labels, imgs, coors)

In [10]:

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # <center> Image Search Framework </center>
    """)
    od_objects = gr.State(None)
    with gr.Column():
        with gr.Row(equal_height=False):
            with gr.Column():
                image = gr.Image(type="pil")
                analyze = gr.Button("Analyze")
                search = gr.Button("Search Full Image")

            with gr.Column():
                img_output = gr.AnnotatedImage()
                gr.Markdown(
                    """
                    # <center> Search By Each Product </center>
                    """)
                
        with gr.Row(equal_height=False):
            gr.Markdown(
                """
                # <center> Before Reranking </center>
                """)
            gallery11 = gr.Gallery(
            label="Search Results", show_label=False, elem_id="gallery"
            ).style(columns=[2], rows=[2], object_fit="contain", height="auto")
            
            gr.Markdown("""
                # <center> Before reranking </center>
                """)
            gallery21 = gr.Gallery(
            label="Search Results", show_label=False, elem_id="gallery"
            ).style(columns=[2], rows=[2], object_fit="contain", height="auto")


                
        with gr.Row():
            gr.Markdown(
            """
            # <center> After Reranking </center>
            """)
            gallery12 = gr.Gallery(
                label="Search Results", show_label=False, elem_id="gallery"
                ).style(columns=[2], rows=[2], object_fit="contain", height="auto")

            gr.Markdown("""
            # <center> After reranking </center>
            """)
            gallery22 = gr.Gallery(
                label="Search Results", show_label=False, elem_id="gallery"
                ).style(columns=[2], rows=[2], object_fit="contain", height="auto")
        search.click(fn=retrieve,
                    inputs=[image],
                    outputs=[gallery12, gallery11])
        analyze.click(products, [image], [img_output, od_objects])
        img_output.select(object_retrieve, [od_objects], [gallery22, gallery21])

demo.launch(share=True)

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://71534f554fdfb11fbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[]
